## **RAG System**

##### Create RAG System to answer questions from pdf document using ollama and langchain

In [1]:
# store document name in variable
PDF_FILE = 'paul.pdf'

# use llama 3.1 
MODEL = 'llama3.1'

### Loading the PDF document

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_FILE)
pages = loader.load()

print(f'Number of pages: {len(pages)}')
print(f'Length of a page: {len(pages[1].page_content)}')
print(f'Content of a page: {pages[1].page_content}')

Number of pages: 9
Length of a page: 3272
Content of a page: 10% a week. And while 110 may not seem much better than 100,
if you keep growing at 10% a week you'll be surprised how big
the numbers get. After a year you'll have 14,000 users, and after
2 years you'll have 2 million.
You'll be doing different things when you're acquiring users a
thousand at a time, and growth has to slow down eventually. But
if the market exists you can usually start by recruiting users
manually and then gradually switch to less manual methods. [3]
Airbnb is a classic example of this technique. Marketplaces are so
hard to get rolling that you should expect to take heroic measures
at first. In Airbnb's case, these consisted of going door to door in
New York, recruiting new users and helping existing ones improve
their listings. When I remember the Airbnbs during YC, I picture
them with rolly bags, because when they showed up for tuesday
dinners they'd always just flown back from somewhere.
Fragile
Airbnb no

### Splitting the pages into chunks

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 1500, chunk_overlap = 100)
chunks = splitter.split_documents(pages)

print(f'Number of chunks:{len(chunks)}')
print(f'Length of a chunk:{len(chunks[1].page_content)}')
print(f'Content of a chunck :{chunks[1].page_content}')

Number of chunks:23
Length of a chunk:1236
Content of a chunck :took better advantage of it than Stripe. At YC we use the term
"Collison installation" for the technique they invented. More
diffident founders ask "Will you try our beta?" and if the answer is
yes, they say "Great, we'll send you a link." But the Collison
brothers weren't going to wait. When anyone agreed to try Stripe
they'd say "Right then, give me your laptop" and set them up on
the spot.
There are two reasons founders resist going out and recruiting
users individually. One is a combination of shyness and laziness.
They'd rather sit at home writing code than go out and talk to a
bunch of strangers and probably be rejected by most of them.
But for a startup to succeed, at least one founder (usually the
CEO) will have to spend a lot of time on sales and marketing. [2]
The other reason founders ignore this path is that the absolute
numbers seem so small at first. This can't be how the big, famous
startups got started, the

### Storing the chunks in a vector store after generate embedding for them

In [6]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OllamaEmbeddings(model = MODEL)
vectorstore = FAISS.from_documents(chunks, embeddings)

/tmp/ipykernel_1752/3573475093.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model = MODEL)


### Setting up a retriever

In [7]:
# how can i get data from my vector store ==> retriever
retriever = vectorstore.as_retriever()
retriever.invoke('What can you get away with when you only have a small number of users')

[Document(id='35b01d71-3bb5-4585-9c2b-a50c1253cba2', metadata={'source': 'paul.pdf', 'page': 4, 'page_label': '5'}, page_content='You can tweak the design faster when you\'re the factory, and you\nlearn things you\'d never have known otherwise. Eric Migicovsky\nof Pebble said one of the things he learned was "how valuable it\nwas to source good screws." Who knew?\nConsult\n8/6/24, 11:04 AM Do Things that Don\'t Scale\nhttps://paulgraham.com/ds.html 5/9'),
 Document(id='082ae8bd-6214-4c29-9d36-d176b093f6ab', metadata={'source': 'paul.pdf', 'page': 7, 'page_label': '8'}, page_content='by calibrating their ambitions, because we know exactly how a lot\nof successful startups looked when they were just getting started.\n[5] If you\'re building something for which you can\'t easily get a\nsmall set of users to observe — e.g. enterprise software — and in\na domain where you have no connections, you\'ll have to rely on\ncold calls and introductions. But should you even be working on\nsuch an i

### Configuring the llm model

In [11]:
from langchain_ollama import ChatOllama

model = ChatOllama(model = MODEL, temperature = 0)
model.invoke('who is the president of the United States')

AIMessage(content="As of my last update in April 2023, Joe Biden is the President of the United States. He was inaugurated as the 46th President on January 20, 2021. However, please note that this information may change over time due to elections or other events.\n\nIf you're looking for more up-to-date information, I recommend checking a reliable news source or official government website for the most current information on the presidency.", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-02-04T04:41:22.308608007Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5007656168, 'load_duration': 2516157144, 'prompt_eval_count': 18, 'prompt_eval_duration': 148000000, 'eval_count': 90, 'eval_duration': 2342000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-9c309dd2-6982-4216-9da9-12ef9f8a6360-0', usage_metadata={'input_tokens': 18, 'output_tokens': 90, 'total_tokens': 108})

### Create a chain and add model & parser to it

In [14]:
# i want to to get rid of AI Message and get my answer as string so we will use string output parser in langchain
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = model | parser

print(chain.invoke('who is the president of the United States'))

As of my last update in April 2023, Joe Biden is the President of the United States. He was inaugurated as the 46th President on January 20, 2021. However, please note that this information may change over time due to elections or other events.

If you're looking for more up-to-date information, I recommend checking a reliable news source or official government website for the most current information on the presidency.


### Setting up a prompt (question & context)


In [16]:
# lets create our prompt template
from langchain.prompts import PromptTemplate

template = """
You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
# show our prompt template
print(prompt.format(context = 'Here is some context', question = 'Here is a question'))


You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: Here is some context

Question: Here is a question



### Adding Prompt to the chain 

In [19]:
chain = prompt | model | parser


# test our chain works fine
chain.invoke({
    'context' : 'Amr\'s brother is medhat',
    'question' : 'who is medhat\'s brother?'
})

'Amr.'

### Adding the retriever to the chain

In [22]:
# Adding retriever to the chain
from operator import itemgetter

chain = (
    {
        "context" : itemgetter('question') | retriever,
        "question" : itemgetter('question')
    }
| prompt
| model
| parser
)

### Use the chain to answer questions

In [23]:
questions = [
    "What can you get away with when you only have a small number of users?",
    "What's the most common unscalable thing founders have to do at the start?",
    "What's one of the biggest things inexperienced founders and investors get wrong about startups?",
]

for question in questions:
       print(f"Question: {question}") 
       print(f"Answers: {chain.invoke({'question':question})}") 
       print('*' * 60) 

Question: What can you get away with when you only have a small number of users?


Answers: You can provide a level of service no big company can.
************************************************************
Question: What's the most common unscalable thing founders have to do at the start?
Answers: Going out and recruiting users individually.
************************************************************
Question: What's one of the biggest things inexperienced founders and investors get wrong about startups?
Answers: They underestimate the power of compound growth.
************************************************************
